In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [4]:
tokenizer.texts_to_sequences(["Firest"])

[[20, 6, 9, 2, 8, 3]]

In [5]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [6]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [7]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [8]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [9]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [10]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [11]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [12]:
dataset = dataset.prefetch(1)

In [ ]:
model = keras.models.Sequential([
                                 keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                                                  dropout=0.2, recurrent_dropout=0.2),
                                 keras.layers.GRU(128, return_sequences=True,
                                                  dropout=0.2, recurrent_dropout=0.2),
                                 keras.layers.TimeDistributed(keras.layers.Dense(max_id, 
                                                                                 activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=20)

Epoch 1/20
     59/Unknown - 23s 398ms/step - loss: 3.1298